<h2 align='center'>Segmenting and Clustering Neighborhoods in Toronto</h2>

We begin by importing the necessary libraries

In [337]:
import numpy as np
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

! pip install lxml
import lxml #to get data from html/websites

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.5
  latest version: 4.9.1

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


Get Data from the webpage

In [338]:
df = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df = pd.DataFrame(df[0])
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


Remove rows with <strong>Not assigned</strong> from Borough column

In [339]:
newdf = df[df['Borough'] != 'Not assigned']
newdf.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Here we replace the rows in the Neighbourhood that contain <strong>Not assigned</strong> with values from Borough column and reset the indices

In [340]:
for value in newdf['Neighbourhood'], newdf['Borough']:
    if (newdf['Neighbourhood'] == 'Not assigned').all():
        newdf['Neighbourhood'] = newdf['Borough']
newdf = newdf.reset_index().drop(labels='index', axis=1)
newdf.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Print the shape to get number of rows

In [341]:
newdf.shape

(103, 3)

Get the Geospatial Data from the given url.

In [342]:
geo_data = pd.read_csv("https://cocl.us/Geospatial_data")
print(geo_data.head(),geo_data.shape)

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476 (103, 3)


Add Longitude and Latitude Columns to the Canada dataset.

In [343]:
canada_data = newdf
canada_data['Latitude'] = geo_data['Latitude']
canada_data['Longitude'] = geo_data['Longitude']
canada_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.806686,-79.194353
1,M4A,North York,Victoria Village,43.784535,-79.160497
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.770992,-79.216917
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.744734,-79.239476
6,M1B,Scarborough,"Malvern, Rouge",43.727929,-79.262029
7,M3B,North York,Don Mills,43.711112,-79.284577
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.716316,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848


Filter Toronto  boroughs to get data for Torornto only

In [344]:
toronto_data = canada_data[canada_data['Borough'].str.contains('Toronto', na=True)]
toronto_data = toronto_data.reset_index().drop(labels='index', axis=1)
toronto_data

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
4,M4E,East Toronto,The Beaches,43.786947,-79.385975
5,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
6,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
7,M6G,Downtown Toronto,Christie,43.753259,-79.329656
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
9,M6H,West Toronto,"Dufferin, Dovercourt Village",43.739015,-79.506944


In [345]:
myhoods = toronto_data[toronto_data['Neighbourhood'].str.contains('')]
latitude = myhoods['Latitude'].values[0]
longitude = myhoods['Longitude'].values[0]
# print(myhoods)
# print(latitude, longitude)

In [346]:
search_query = ''
radius = 500
print(search_query + ' .... OK!')

 .... OK!


In [347]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
# url

In [348]:
results = requests.get(url).json()
# results

In [349]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = pd.json_normalize(venues)
# dataframe.head()

In [350]:
# set(dataframe.name)

In [351]:
# venues
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are \n{}, \n{}, {}.'.format(location,latitude, longitude))

The geograpical coordinate of Toronto are 
Toronto, Golden Horseshoe, Ontario, Canada, 
43.6534817, -79.3839347.


In [352]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, myhoods in zip(myhoods['Latitude'], myhoods['Longitude'], myhoods['Borough'], myhoods['Neighbourhood']):
    label = '{}, {}, {}'.format(myhoods, borough, 'Canada')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
# map_newyork

In [353]:
map_newyork

In [354]:
downtownhoods = toronto_data[toronto_data['Borough'].str.contains('Downtown Toronto')] 
latitude = downtownhoods['Latitude'].values[0]
longitude = downtownhoods['Longitude'].values[0]
mydowntownhoods = downtownhoods.copy()
downtownhoods
# print(latitude, longitude)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389
5,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714
6,M5G,Downtown Toronto,Central Bay Street,43.782736,-79.442259
7,M6G,Downtown Toronto,Christie,43.753259,-79.329656
8,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.737473,-79.464763
10,M5J,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",43.695344,-79.318389
13,M5K,Downtown Toronto,"Toronto Dominion Centre, Design Exchange",43.668999,-79.315572


In [355]:
# venues
address = 'Downtown Toronto, Canada'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are \n{}, \n{}, {}.'.format(location,latitude, longitude))

The geograpical coordinate of Downtown Toronto are 
Downtown Yonge, Old Toronto, Toronto, Golden Horseshoe, Ontario, M5B 2C3, Canada, 
43.6563221, -79.3809161.


In [356]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, downtownhoods in zip(downtownhoods['Latitude'], downtownhoods['Longitude'], downtownhoods['Borough'], downtownhoods['Neighbourhood']):
    label = '{}, {}, {}'.format(downtownhoods, borough, 'Canada')
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
# map_newyork

In [357]:
map_newyork

In [358]:
CLIENT_ID = 'LVBTQT01Y1TPOBRQ2L1GLN5RJM33CWYS5I24KFBDCOCRVY0E' # your Foursquare ID
CLIENT_SECRET = 'GLPR51V3LUG1OSYLDI25KMIXFM0JQU0RVEI5TUVVIJMLGHBS' # your Foursquare Secret
VERSION = '202010278'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: LVBTQT01Y1TPOBRQ2L1GLN5RJM33CWYS5I24KFBDCOCRVY0E
CLIENT_SECRET:GLPR51V3LUG1OSYLDI25KMIXFM0JQU0RVEI5TUVVIJMLGHBS


In [359]:
mydowntownhoods.loc[0, 'Neighbourhood']

'Regent Park, Harbourfront'

In [360]:
neighborhood_latitude = mydowntownhoods.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = mydowntownhoods.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = mydowntownhoods.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Regent Park, Harbourfront are 43.7635726, -79.1887115.


In [361]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=LVBTQT01Y1TPOBRQ2L1GLN5RJM33CWYS5I24KFBDCOCRVY0E&client_secret=GLPR51V3LUG1OSYLDI25KMIXFM0JQU0RVEI5TUVVIJMLGHBS&v=202010278&ll=43.7635726,-79.1887115&radius=500&limit=100'

In [362]:
results = requests.get(url).json()

In [363]:
pd.json_normalize(results['response']['groups'][0]['items']).head()

,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,venue.location.distance,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups
0,e-0-4beee041e24d20a1cd857314-0,0,"[{'summary': 'This spot is popular', 'type': '...",4beee041e24d20a1cd857314,RBC Royal Bank,4374 KINGSTON RD,Kingston & Lawrence,43.766790,-79.191151,"[{'label': 'display', 'lat': 43.76678992471017...",408,M1E 2M8,CA,Scarborough,ON,Canada,"[4374 KINGSTON RD (Kingston & Lawrence), Scarb...","[{'id': '4bf58dd8d48988d10a951735', 'name': 'B...",0,[]
1,e-0-4c62f34bde1b2d7fec89e370-1,0,"[{'summary': 'This spot is popular', 'type': '...",4c62f34bde1b2d7fec89e370,G & G Electronics,4371 kingston road,NaN,43.765309,-79.191537,"[{'label': 'display', 'lat': 43.765309, 'lng':...",298,M1E 2M9,CA,Toronto,ON,Canada,"[4371 kingston road, Toronto ON M1E 2M9, Canada]","[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",0,[]
2,e-0-57fd24f6cd1083addfd77bf9-2,0,"[{'summary': 'This spot is popular', 'type': '...",57fd24f6cd1083addfd77bf9,Sail Sushi,9-4352 Kingston Rd,NaN,43.765951,-79.191275,"[{'label': 'display', 'lat': 43.765951, 'lng':...",335,M1E 2M8,CA,Scarborough,ON,Canada,"[9-4352 Kingston Rd, Scarborough ON M1E 2M8, C...","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[]
3,e-0-5411f741498e9ebd5e35d8bd-3,0,"[{'summary': 'This spot is popular', 'type': '...",5411f741498e9ebd5e35d8bd,Big Bite Burrito,4383 Kingston rd.,NaN,43.766299,-79.190720,"[{'label': 'display', 'lat': 43.76629908447079...",343,NaN,CA,Scarborough,ON,Canada,"[4383 Kingston rd., Scarborough ON, Canada]","[{'id': '4bf58dd8d48988d1c1941735', 'name': 'M...",0,[]
4,e-0-4c1c7f9bb306c9288f0464b7-4,0,"[{'summary': 'This spot is popular', 'type': '...",4c1c7f9bb306c9288f0464b7,Enterprise Rent-A-Car,4304 Kingston Rd,NaN,43.764076,-79.193406,"[{'label': 'display', 'lat': 43.7640757, 'lng'...",381,M1E 2M8,CA,Scarborough,ON,Canada,"[4304 Kingston Rd, Scarborough ON M1E 2M8, Can...","[{'id': '4bf58dd8d48988d1ef941735', 'name': 'R...",0,[]


In [364]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [365]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,RBC Royal Bank,Bank,43.766790,-79.191151
1,G & G Electronics,Electronics Store,43.765309,-79.191537
2,Sail Sushi,Restaurant,43.765951,-79.191275
3,Big Bite Burrito,Mexican Restaurant,43.766299,-79.190720
4,Enterprise Rent-A-Car,Rental Car Location,43.764076,-79.193406


In [366]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

8 venues were returned by Foursquare.


In [367]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [368]:
Downtown_venues = getNearbyVenues(names=mydowntownhoods['Neighbourhood'], latitudes=mydowntownhoods['Latitude'], longitudes=mydowntownhoods['Longitude'])
Downtown_venues.head()

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Stn A PO Boxes
St. James Town, Cabbagetown
First Canadian Place, Underground city
Church and Wellesley


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.763573,-79.188711,RBC Royal Bank,43.766790,-79.191151,Bank
1,"Regent Park, Harbourfront",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Regent Park, Harbourfront",43.763573,-79.188711,Sail Sushi,43.765951,-79.191275,Restaurant
3,"Regent Park, Harbourfront",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
4,"Regent Park, Harbourfront",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location


In [369]:
Downtown_venues.shape

(151, 7)

In [370]:
Downtown_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Berczy Park,1,1,1,1,1,1
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",16,16,16,16,16,16
Central Bay Street,5,5,5,5,5,5
Christie,2,2,2,2,2,2
Church and Wellesley,6,6,6,6,6,6
"Commerce Court, Victoria Hotel",2,2,2,2,2,2
"First Canadian Place, Underground city",1,1,1,1,1,1
"Garden District, Ryerson",4,4,4,4,4,4
"Harbourfront East, Union Station, Toronto Islands",7,7,7,7,7,7


In [371]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 85 uniques categories.


## 3. Analyze Each Neighborhood


In [372]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_onehot['Neighbourhood'] = Downtown_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]

Downtown_onehot.head()

,Neighbourhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Comic Shop,Construction & Landscaping,Convenience Store,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health Food Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Kids Store,Latin American Restaurant,Light Rail Station,Liquor Store,Martial Arts School,Medical Center,Mexican Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Recording Studio,Rental Car Location,Restaurant,Sandwich Place,Skate Park,Skating Rink,Smoothie Shop,Social Club,Steakhouse,Supplement Shop,Sushi Restaurant,Tanning Salon,Thai Restaurant,Thrift / Vintage Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


And let's examine the new dataframe size.


In [373]:
Downtown_onehot.shape

(151, 86)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category


In [374]:
Downtown_grouped = Downtown_onehot.groupby('Neighbourhood').mean().reset_index()
Downtown_grouped

,Neighbourhood,Airport,Athletics & Sports,Auto Workshop,Bakery,Bank,Bar,Baseball Field,Beer Store,Bookstore,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Butcher,Café,Caribbean Restaurant,Chinese Restaurant,Coffee Shop,College Stadium,Comic Shop,Construction & Landscaping,Convenience Store,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Food & Drink Shop,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,General Entertainment,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Hakka Restaurant,Hardware Store,Health Food Store,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Intersection,Italian Restaurant,Kids Store,Latin American Restaurant,Light Rail Station,Liquor Store,Martial Arts School,Medical Center,Mexican Restaurant,Movie Theater,Park,Pet Store,Pharmacy,Pizza Place,Playground,Pub,Recording Studio,Rental Car Location,Restaurant,Sandwich Place,Skate Park,Skating Rink,Smoothie Shop,Social Club,Steakhouse,Supplement Shop,Sushi Restaurant,Tanning Salon,Thai Restaurant,Thrift / Vintage Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Wings Joint,Yoga Studio
0,Berczy Park,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.00,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.0000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,1.0,0.000,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.0,0.000000,0.0625,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.062500,0.0000,0.062500,0.0625,0.000000,0.000,0.000000,0.000000,0.00,0.0625,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0625,0.062500,0.000000,0.0,0.000000,0.000,0.0,0.0625,0.0625,0.000,0.00,0.000000,0.000000,0.000000,0.0625,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0625,0.000000,0.0,0.000,0.000,0.000000,0.062500,0.000000,0.000000,0.062500,0.0,0.000000,0.0625,0.000,0.062500,0.000000,0.0625,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
2,Central Bay Street,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.2000,0.000000,0.000,0.000000,0.200000,0.00,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.0,0.000000,0.000,0.0,0.0000,0.0000,0.000,0.00,0.000000,0.200000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0,0.000,0.000,0.000000,0.000000,0.000000,0.200000,0.200000,0.0,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
3,Christie,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.000,0.000000,0.000000,0.00,0.0000,0.0,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.0000,0.000000,0.000000,0.5,0.000000,0.000,0.0,0.0000,0.0000,0.000,0.00,0.000000,0.000000,0.000000,0.0000,0.000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0,0.000,0.000,0.000000,0.500000,0.000000,0.000000,0.000000,0.0,0.000000,0.0000,0.000,0.000000,0.000000,0.0000,0.000000,0.000000,0.0000,0.000000,0.0000,0.000000,0.0000,0.000,0.0000,0.0,0.0,0.000000,0.0000,0.000000
4,Church and Wellesley,0.0,0.000000,0.0000,0.0000,0.000000,0.000000,0.0,0.000

#### Let's confirm the new size


In [375]:
Downtown_grouped.shape

(19, 86)

#### Let's print each neighborhood along with the top 5 most common venues


In [376]:
num_top_venues = 5

for hood in Downtown_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                 venue  freq
0  Martial Arts School   1.0
1              Airport   0.0
2   Light Rail Station   0.0
3             Pharmacy   0.0
4            Pet Store   0.0


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                  venue  freq
0         Garden Center  0.06
1         Burrito Place  0.06
2        Farmers Market  0.06
3  Fast Food Restaurant  0.06
4      Recording Studio  0.06


----Central Bay Street----
           venue  freq
0    Pizza Place   0.2
1       Pharmacy   0.2
2  Grocery Store   0.2
3    Coffee Shop   0.2
4        Butcher   0.2


----Christie----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5
2            Airport   0.0
3          Pet Store   0.0
4      Movie Theater   0.0


----Church and Wellesley----
                venue  freq
0      Discount Store  0.17
1      Sandwich Place  0.17
2        Intersection  0.17
3         Coffee S

#### Let's put that into a _pandas_ dataframe


First, let's write a function to sort the venues in descending order.


In [377]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.


In [378]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Downtown_grouped['Neighbourhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Martial Arts School,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
1,"CN Tower, King and Spadina, Railway Lands, Har...",Garden Center,Brewery,Light Rail Station,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place
2,Central Bay Street,Pharmacy,Butcher,Pizza Place,Coffee Shop,Grocery Store,Bank,Fish & Chips Shop,Dessert Shop,Diner,Discount Store
3,Christie,Park,Food & Drink Shop,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant
4,Church and Wellesley,Pizza Place,Coffee Shop,Chinese Restaurant,Discount Store,Sandwich Place,Intersection,Yoga Studio,Falafel Restaurant,Dance Studio,Dessert Shop


<a id='item4'></a>


## 4. Cluster Neighborhoods


Run _k_-means to cluster the neighborhood into 5 clusters.


In [379]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 1, 1, 2, 1, 4, 3, 1, 1, 1], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.


In [380]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Downtown_merged = mydowntownhoods

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
Downtown_merged = Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Downtown_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.763573,-79.188711,1,Mexican Restaurant,Intersection,Medical Center,Bank,Rental Car Location,Restaurant,Breakfast Spot,Electronics Store,Dance Studio,Dessert Shop
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.773136,-79.239476,1,Athletics & Sports,Gas Station,Bakery,Bank,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Yoga Studio,Farmers Market
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.692657,-79.264848,1,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Farmers Market,Dance Studio,Dessert Shop,Diner,Discount Store
3,M5C,Downtown Toronto,St. James Town,43.799525,-79.318389,1,Fast Food Restaurant,Gym,Breakfast Spot,Grocery Store,Indian Restaurant,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Pharmacy
5,M5E,Downtown Toronto,Berczy Park,43.757490,-79.374714,0,Martial Arts School,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market


Finally, let's visualize the resulting clusters


In [381]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighbourhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<a id='item5'></a>


## 5. Examine Clusters


Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster. Based on the defining categories, you can then assign a name to each cluster. I will leave this exercise to you.


#### Cluster 1


In [382]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 0, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Downtown Toronto,0,Martial Arts School,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market


In [383]:
Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]

Index(['Borough', 'Cluster Labels', '1st Most Common Venue',
       '2nd Most Common Venue', '3rd Most Common Venue',
       '4th Most Common Venue', '5th Most Common Venue',
       '6th Most Common Venue', '7th Most Common Venue',
       '8th Most Common Venue', '9th Most Common Venue',
       '10th Most Common Venue'],
      dtype='object')

In [384]:
range(5, Downtown_merged.shape[1])

range(5, 16)

#### Cluster 2


In [385]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 1, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Mexican Restaurant,Intersection,Medical Center,Bank,Rental Car Location,Restaurant,Breakfast Spot,Electronics Store,Dance Studio,Dessert Shop
1,Downtown Toronto,1,Athletics & Sports,Gas Station,Bakery,Bank,Thai Restaurant,Hakka Restaurant,Fried Chicken Joint,Caribbean Restaurant,Yoga Studio,Farmers Market
2,Downtown Toronto,1,General Entertainment,College Stadium,Café,Skating Rink,Yoga Studio,Farmers Market,Dance Studio,Dessert Shop,Diner,Discount Store
3,Downtown Toronto,1,Fast Food Restaurant,Gym,Breakfast Spot,Grocery Store,Indian Restaurant,Pizza Place,Coffee Shop,Chinese Restaurant,Sandwich Place,Pharmacy
6,Downtown Toronto,1,Pharmacy,Butcher,Pizza Place,Coffee Shop,Grocery Store,Bank,Fish & Chips Shop,Dessert Shop,Diner,Discount Store
10,Downtown Toronto,1,Skating Rink,Park,Athletics & Sports,Curling Ice,Dance Studio,Beer Store,Dessert Shop,Diner,Discount Store,Electronics Store
13,Downtown Toronto,1,Park,Sandwich Place,Fish & Chips Shop,Fast Food Restaurant,Ice Cream Shop,Italian Restaurant,Liquor Store,Coffee Shop,Movie Theater,Pet Store
27,Downtown Toronto,1,Turkish Restaurant,Bar,Discount Store,Restaurant,Sandwich Place,Yoga Studio,Farmers Market,Curling Ice,Dance Studio,Dessert Shop
30,Downtown Toronto,1,Café,Coffee Shop,Pub,Italian Restaurant,Sushi Restaurant,Pizza Place,Fish & Chips Shop,Latin American Restaurant,Indie Movie Theater,Health Food Store
32,Downtown Toronto,1,Garden Center,Brewery,Light Rail Station,Comic Shop,Park,Pizza Place,Recording Studio,Restaurant,Butcher,Burrito Place


#### Cluster 3


In [386]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 2, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Downtown Toronto,2,Park,Food & Drink Shop,Yoga Studio,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant
8,Downtown Toronto,2,Airport,Park,Fast Food Restaurant,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market


#### Cluster 4


In [387]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 3, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,Downtown Toronto,3,Construction & Landscaping,Baseball Field,Fish & Chips Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market
36,Downtown Toronto,3,Baseball Field,Yoga Studio,Fish & Chips Shop,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store,Falafel Restaurant,Farmers Market


#### Cluster 5


In [388]:
Downtown_merged.loc[Downtown_merged['Cluster Labels'] == 4, Downtown_merged.columns[[1] + list(range(5, Downtown_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Downtown Toronto,4,Playground,Trail,Yoga Studio,Farmers Market,Curling Ice,Dance Studio,Dessert Shop,Diner,Discount Store,Electronics Store
